In [ ]:
#!pip install summerepi2==1.3.5

In [ ]:
import pandas as pd
from plotly import express as px
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def get_si_base_structure(
    extra_comps: list = None,
    model_duration: float = 40.0
) -> CompartmentalModel:
    """
    Generate a model that only has S and I compartments, but has the basic
    characteristics that we can then use to add in different assumptions
    around post-infection immunity.
    
    Args:
        extra_comps: Any compartments to incorporate in addition to the base ones
    Returns:
        The summer model object
    """
    
    # Compartments are comprised of the base ones and any additional latency compartments requested
    if extra_comps is None:
        extra_comps = []
    
    compartments = ["susceptible", "infectious"] + extra_comps
    
    infectious_compartment = ("infectious",)
    analysis_times = (0.0, model_duration)
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
    )

    seed_prop = 0.001

    model.set_initial_population(
        {
            "susceptible": 1.0 - seed_prop,
            "infectious": seed_prop,
        }
    )
    
    model.add_infection_frequency_flow(
        "infection", 
        Parameter("contact_rate"),
        "susceptible", 
        "infectious",
    )
    
    return model

In [ ]:
parameters = {
    "contact_rate": 1.0,
    "recovery_rate": 0.333,
    "death_rate": 0.05,
}

### SI structure
We can represent permanent infection and infectiousness
by ensuring that anyone entering the `infectious` compartment
remains forever trapped within this state.
Fortunately, there are relatively few infections
that would be well represented by this model structure.
Although pathogens exist that result in permanent infection,
there are not many that also render the host permanently infectious.

In [ ]:
si_model = get_si_base_structure()

si_model.run(parameters)
si_values = si_model.get_outputs_df()
axis_labels = {"index": "time", "value": "proportion"}
si_values.plot(labels=axis_labels)

### SIR structure
As a starting point, let's next consider a model 
that is more similar to the SEIR structure that we have been
using throughout several of the preceding notebooks.
Here we assume that immunity to reinfection is 
permanent and complete, which may be appropriate
for some infectious diseases.

In [ ]:
sir_model = get_si_base_structure(["recovered"])

sir_model.add_transition_flow("recovery", Parameter("recovery_rate"), "infectious", "recovered")

sir_model.run(parameters)
sir_values = sir_model.get_outputs_df()
sir_values.plot(labels=axis_labels)

### SIS structure
Under this structure,
no immunity is conferred by infection.
That is, recovered individuals are at the same
risk of reinfection as those who have never been infected.

In [ ]:
sis_model = get_si_base_structure()

sis_model.add_transition_flow("recovery", Parameter("recovery_rate"), "infectious", "susceptible")

sis_model.run(parameters)
sis_values = sis_model.get_outputs_df()
sis_values.plot(labels=axis_labels)

### SIRS structure
Under this assumption,
immunity is obtained after recovery from the infectious state, 
but only for a limited period.
After an initial epidemic wave depletes the susceptible population,
the model approaches an equilibrium state in which
the rate of infection offsets the rate of waning of immunity
from the recovered population.

In [ ]:
sirs_model = get_si_base_structure(["recovered"])

sirs_model.add_transition_flow("recovery", Parameter("recovery_rate"), "infectious", "recovered")
sirs_model.add_transition_flow("immunity_waning", Parameter("immunity_waning"), "recovered", "susceptible")

parameters_waning = parameters | {"immunity_waning": 0.1}

sirs_model.run(parameters_waning)
sirs_values = sirs_model.get_outputs_df()
sirs_values.plot(labels=axis_labels)

## Tracking immunity status
One important consideration whenever constructing
compartmental models of infectious disease transmission
is that these models are "memory-less".
That is, the model state at future time points are entirely
determined by the model's current state.
Therefore, although we can calculate the rate of 
new persons transitioning between two given model states
at a certain point in time,
calculating these rates does not provide us with information
about the history of the new arrivals into the destination compartment.
If we wish to obtain this sort of information from a compartmental model,
this can be achieved by incorporating additional compartments to track
past states.
For example, consider an alternative structure to the SIRS
assumption around waning immunity.

In [ ]:
sirs2_model = get_si_base_structure(["recovered", "susceptible_2"])

sirs2_model.add_infection_frequency_flow("reinfection", Parameter("contact_rate"), "susceptible_2", "infectious") 
sirs2_model.add_transition_flow("recovery", Parameter("recovery_rate"), "infectious", "recovered")
sirs2_model.add_transition_flow("immunity_waning", Parameter("immunity_waning"), "recovered", "susceptible_2")

sirs2_model.run(parameters_waning)
sirs2_values = sirs2_model.get_outputs_df()
sirs2_values.plot(labels=axis_labels)

Note that the `susceptible_2` compartment behaves exactly 
the same as the `susceptible` compartment from the point-of-view 
of the model, but has a different history.
We might have alternatively called this compartment `waned`.
Note that the dynamics of this model are identical
to those of the SIRS model in which recovery
returned infectious persons to the starting susceptible compartment. 
We could now produce outputs for quantities including 
the proportion of the total population ever infected 
and the proportion of infections attributable to reinfection.
These quantities could not have been obtained from
the simpler SIRS model introduced above.

## Comparison
Let's confirm that this is the case and that 
the models have the same behaviour.
(Toggle the SIRS2 line off to see the SIRS line underneath.)

In [ ]:
pd.DataFrame(
    {
        "si": si_values["infectious"],
        "sir": sir_values["infectious"],
        "sis": sis_values["infectious"],
        "sirs": sirs_values["infectious"],
        "sirs2": sirs2_values["infectious"],
    }
).plot(labels=axis_labels)

### Oscillatory dynamics

In [ ]:
sirs_long = get_si_base_structure(["recovered"], 1000.0)

sirs_long.add_transition_flow("recovery", Parameter("recovery_rate"), "infectious", "recovered")
sirs_long.add_transition_flow("immunity_waning", Parameter("immunity_waning"), "recovered", "susceptible")

parameters_waning = parameters | {"immunity_waning": 0.01}

sirs_long.run(parameters_waning)

outputs = sirs_long.get_outputs_df()

outputs.plot()

### Phase plane
We've examined these epidemics by looking at the compartment sizes over time. However, this process of oscillatory dynamics heading gradually towards a stable endemic state at which the recovered and the susceptible populations balance one another can be illustrated in other ways.

An alternative way to think about what is happening in the model is through a "phase plane" in which we plot the susceptible and infectious populations against one another. This gives a nice sense of the endemic state as a stable equilibrium point that is attracting the epidemic towards it at any point in time. However, the momentum of the epidemic starts off so great that it continually overshoots this stable point and ends up spiralling towards it's final resting place. You can imagine it as a bit like a coin wishing well, with the time dimension approximately represented by the distance along the spiral.

In [ ]:
outputs_late_start = outputs.loc[outputs.index > 70.0]

fig = px.line(
    outputs_late_start,
    x="susceptible", 
    y="infectious"
)
fig.show()

In [ ]:
px.line_3d(
    outputs_late_start, 
    x="susceptible", 
    y="infectious", 
    z=outputs_late_start.index,
    width=800,
    height=600,
)